# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("../output_data/WeatherPy.csv")

weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,hermanus,-34.4187,19.2345,52.81,66,46,5.08,ZA,1627333204
1,msanga,-6.0500,36.0333,60.71,55,62,5.73,TZ,1627333205
2,rutul,41.5357,47.4239,62.51,91,0,0.16,RU,1627333206
3,hithadhoo,-0.6000,73.0833,80.20,80,95,7.70,MV,1627333207
4,kendari,-3.9450,122.4989,70.12,93,32,2.91,ID,1627333208


In [3]:
weather_df.describe()

,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,585.000000,585.000000,585.000000,585.000000,585.000000,585.000000,5.850000e+02
mean,18.923349,15.301728,70.694991,69.131624,55.203419,7.925538,1.627333e+09
std,32.991284,88.930882,14.541740,21.899946,38.378406,5.529200,8.160994e+01
min,-54.800000,-179.166700,30.900000,7.000000,0.000000,0.000000,1.627333e+09
25%,-8.116700,-60.673300,60.710000,56.000000,16.000000,4.000000,1.627333e+09
50%,20.516900,16.531200,71.490000,76.000000,63.000000,6.910000,1.627333e+09
75%,46.632800,90.092800,80.730000,85.000000,95.000000,10.510000,1.627333e+09
max,78.218600,179.316700,107.280000,100.000000,100.000000,35.500000,1.627333e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)
loc = weather_df[["Lat", "Lng"]]
humidity = weather_df['Humidity']

In [5]:
# Plot Heatmap
fig = gmaps.figure(center=(45.0, -5.0), zoom_level=2)
intensity = np.max(humidity)

# Create heat layer
heat = gmaps.heatmap_layer(loc, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
intensity

100

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
new_df = weather_df.loc[(weather_df["Wind Speed"] <= 10) & (weather_df["Cloudiness"] == 0) & \
                                   (weather_df["Max Temp"] >= 70) & (weather_df["Max Temp"] <= 80)].dropna()


new_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
91,ostersund,63.1792,14.6357,73.85,43,0,9.22,SE,1627333246
104,uhlove,44.8215,33.6045,77.58,64,0,9.57,UA,1627333250
163,kula,38.5473,28.6498,72.61,48,0,3.40,TR,1627333270
316,fukue,32.6881,128.8419,78.53,79,0,3.76,JP,1627333315
320,palma,39.5694,2.6502,77.05,51,0,3.00,ES,1627333317
375,olinda,-8.0089,-34.8553,74.93,78,0,9.22,BR,1627333332
382,urumqi,43.8010,87.6005,79.75,23,0,8.95,CN,1627333065
458,nador,35.1740,-2.9287,71.85,78,0,3.44,MA,1627333354
472,garca,-22.2106,-49.6561,76.08,33,0,2.10,BR,1627333307
542,indramayu,-6.3264,108.3200,75.42,81,0,7.90,ID,1627333378


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = new_df.loc[:,["Lat","Lng","City","Country"]]
hotel_df["Hotel Name"] = ""
hotel_df

,Lat,Lng,City,Country,Hotel Name
91,63.1792,14.6357,ostersund,SE,
104,44.8215,33.6045,uhlove,UA,
163,38.5473,28.6498,kula,TR,
316,32.6881,128.8419,fukue,JP,
320,39.5694,2.6502,palma,ES,
375,-8.0089,-34.8553,olinda,BR,
382,43.8010,87.6005,urumqi,CN,
458,35.1740,-2.9287,nador,MA,
472,-22.2106,-49.6561,garca,BR,
542,-6.3264,108.3200,indramayu,ID,


In [9]:
api = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [10]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(api, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 91: ostersund.
Closest hotel in ostersund is STF Östersund Ledkrysset hostel.
------------
Retrieving Results for Index 104: uhlove.
Closest hotel in uhlove is Our Sail.
------------
Retrieving Results for Index 163: kula.
Closest hotel in kula is Anemon Kula Hotel.
------------
Retrieving Results for Index 316: fukue.
Closest hotel in fukue is Campana Hotel.
------------
Retrieving Results for Index 320: palma.
Closest hotel in palma is Castillo Hotel Son Vida, a Luxury Collection Hotel, Mallorca.
------------
Retrieving Results for Index 375: olinda.
Closest hotel in olinda is Hotel 7 Colinas.
------------
Retrieving Results for Index 382: urumqi.
Closest hotel in urumqi is Sheraton Urumqi Hotel.
------------
Retrieving Results for Index 458: nador.
Closest hotel in nador is Hôtel Mercure Rif Nador.
------------
Retrieving Results for Index 472: garca.
Closest hotel in garca is Meu Hotel Garça.
------------
Retrieving Results for Index 542: indramayu.
Clo

In [11]:
hotel_df

,Lat,Lng,City,Country,Hotel Name
91,63.1792,14.6357,ostersund,SE,STF Östersund Ledkrysset hostel
104,44.8215,33.6045,uhlove,UA,Our Sail
163,38.5473,28.6498,kula,TR,Anemon Kula Hotel
316,32.6881,128.8419,fukue,JP,Campana Hotel
320,39.5694,2.6502,palma,ES,"Castillo Hotel Son Vida, a Luxury Collection H..."
375,-8.0089,-34.8553,olinda,BR,Hotel 7 Colinas
382,43.8010,87.6005,urumqi,CN,Sheraton Urumqi Hotel
458,35.1740,-2.9287,nador,MA,Hôtel Mercure Rif Nador
472,-22.2106,-49.6561,garca,BR,Meu Hotel Garça
542,-6.3264,108.3200,indramayu,ID,Grand Trisula Hotel


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
marker = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))